In [ ]:
#Helps split image into 2 separate files with eyes in image
import cv2

face_cascade = cv2.CascadeClassifier('D:/deep/haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier('D:/deep/haarcascade_eye.xml')

# Saves eyes into the same directory, except with preceding "left_eye_" or "right_eye_"
def getEyesFromFilename (imgDir,imgName):

    imgLocation = imgDir + imgName
    img = cv2.imread(imgLocation)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    left_eye_found = False
    right_eye_found = False
    
    faces = face_cascade.detectMultiScale(gray, 1.05, 5)
    for (x,y,w,h) in faces:
        img = cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = img[y:y+h, x:x+w]
        eyes = eye_cascade.detectMultiScale(roi_gray)
        
        n = 0

        for (ex,ey,ew,eh) in eyes:
            cv2.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(0,255,0),2)
            crop_img = roi_gray[ey:ey + eh, ex:ex + ew]

            if n is 0:
                cv2.imwrite(imgDir+"left-eye-"+imgName,crop_img)
                left_eye_found = True

            elif n is 1:
                cv2.imwrite(imgDir+"right-eye-"+imgName,crop_img)
                right_eye_found = True
                
            n+=1
    
    return (left_eye_found, right_eye_found)
      
     

def getImageAsList(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    for (x, y, w, h) in faces:
        img = cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 0), 2)
        roi_gray = gray[y:y + h, x:x + w]
        roi_color = img[y:y + h, x:x + w]
        eyes = eye_cascade.detectMultiScale(roi_gray)

        eyeArray = []

        for (ex, ey, ew, eh) in eyes:
            cv2.rectangle(roi_color, (ex, ey), (ex + ew, ey + eh), (0, 255, 0), 2)
            crop_img = roi_gray[ey:ey + eh, ex:ex + ew]
            eyeArray.append(crop_img)

        return eyeArray

In [82]:
import keras
from PIL import Image
from keras.models import load_model
import numpy as np

#Load a Keras model from a .h5 file
model = load_model('C:/Users/sahil/Desktop/Files/Coding/projects/classifeye/cassifeye.h5')


def getPredictionFromEye(image_path, image_name):
    img = Image.open(image_path+image_name)
    img = img.resize((25,25),resample=0)
    numpyAr = np.array(img).reshape(1,25,25,1)
    if(model.predict(x=numpyAr)>.5):
        return True
    else:
        return False


def getPredicationFromImg(image_path, image_name):
    eyes = getImageAsList(cv2.imread(image_path+ image_name))
    num_eyes = 0
    awake = False

    if eyes != None:
        num_eyes = len(eyes)
    else:
        return 2
        
    for i in range(num_eyes):
        if not awake:
            cv2.imwrite(image_path+'testing-'+image_name, eyes[i])
            if getPredictionFromEye(image_path, 'testing-'+image_name):
                awake = True
                
    if awake:
        return 1
    else:
        return 0
    

In [83]:
print(getPredicationFromImg('D:/deep/dataset_B_FacialImages/','lebron.jpg'))

1
